## Convert files from Thermo

Download proteowizard (Windows only) https://proteowizard.sourceforge.io/tools/msconvert.html

Run the following command:

msconvert *.raw --filter "peakPicking true 1- " --filter "msLevel 1-" --ignoreUnknownInstrumentError

Or PeakPicking MS 1- through the GUI

## Import libraries

In [21]:
from pyopenms import *
from pandas import DataFrame
import pandas as pd
import pyteomics
from pyteomics.openms import featurexml
import numpy as np
import sys
from pyteomics import mztab

## Preprocessing step

The first preprocessing function consists of six steps:

### 1) PrecursorCorrection (To the "highest intensity MS1 peak")

So the highestintensity is used directly right after the file introduction, in order to correct wrong MS1 precursor annotation - this means it will correct in the precursor window (e.g. 0.01 Da) and then use the highest intensity peak as the precursor. This is to correct for any instrument error. Since we assume that in the give mass window, the precursor with the hightest intensity was actually fragmented instead of any other (top-n method).

### 2) Mass trace detection

A mass trace extraction method that gathers peaks similar in m/z and moving along retention time.

Peaks of a MSExperiment are sorted by their intensity and stored in a list of potential chromatographic apex positions. Only peaks that are above the noise threshold (user-defined) are analyzed and only peaks that are n times above this minimal threshold are considered as apices. This saves computational resources and decreases the noise in the resulting output.

Starting with these, mass traces are extended in- and decreasingly in retention time. During this extension phase, the centroid m/z is computed on-line as an intensity-weighted mean of peaks.

The extension phase ends when either the frequency of gathered peaks drops below a threshold (min_sample_rate, see MassTraceDetection parameters) or when the number of missed scans exceeds a threshold (trace_termination_outliers, see MassTraceDetection parameters).

Finally, only mass traces that pass a filter (a certain minimal and maximal length as well as having the minimal sample rate criterion fulfilled) get added to the result.

### 3) Elution peak detection

Extracts chromatographic peaks from a mass trace.

Mass traces may consist of several consecutively (partly overlapping) eluting peaks, e.g., stemming from (almost) isobaric compounds that are separated by retention time. Especially in metabolomics, isomeric compounds with exactly the same mass but different retentional behaviour may still be contained in the same mass trace.

This method first applies smoothing on the mass trace's intensities, then detects local minima/maxima in order to separate the chromatographic peaks from each other. Detection of maxima is performed on the smoothed intensities and uses a fixed peak width (given as parameter chrom_fwhm) within which only a single maximum is expected. Currently smoothing is done using SavitzkyGolay smoothing with a second order polynomial and a frame length of the fixed peak width.

Depending on the "width_filtering" parameters, mass traces are filtered by length in seconds ("fixed" filter) or by quantile.

The output of the algorithm is a set of chromatographic peaks for each mass trace, i.e. a vector of split mass traces (see ElutionPeakDetection parameters).

In general, a user would want to call the "detectPeaks" functions, potentially followed by the "filterByPeakWidth" function.

### 4) Feature detection

FeatureFinderMetabo assembles metabolite features from singleton mass traces.

Mass traces alone would allow for further analysis such as metabolite ID or statistical evaluation. However, in general, monoisotopic mass traces are accompanied by satellite C13 peaks and thus may render the analysis more difficult. FeatureFinderMetabo fulfills a further data reduction step by assembling compatible mass traces to metabolite features (that is, all mass traces originating from one metabolite). To this end, multiple metabolite hypotheses are formulated and scored according to how well differences in RT (optional), m/z or intensity ratios match to those of theoretical isotope patterns.

If the raw data scans contain the scan polarity information, it is stored as meta value "scan_polarity" in the output file.

Mass trace clustering can be done using either 13C distances or a linear model (Kenar et al) – see parameter 'ffm:mz_scoring_13C'. Generally, for lipidomics, use 13C, since lipids contain a lot of 13C. For general metabolites, the linear model is usually more appropriate. To decide what is better, the total number of features can be used as indirect measure.

the lower(!) the better (since more mass traces are assembled into single features). Detailed information is stored in the featureXML output: it contains meta-values for each feature about the mass trace differences (inspectable via TOPPView). 

By default, the linear model is used.

### 5) Metabolite adduct decharger (MetaboliteFeatureDeconvolution)

For each peak, this algorithm reconstructs neutral masses by enumerating all possible adducts with matching charge. 
You can add the list of adduct files and database files for the algorithm to parse through.
With SIRIUS, an algorithm that is later used, you are only able to use singly charged adducts so charges higher than 1 are filtered out.

### 6) PrecursorCorrection (To the "nearest feature”)

This algorithm is used after feature detection, adduct grouping and even identification via accurate mass search. It basically allows the precursor correction on MS2 level. Which means that if there are MS2 spectra in my feature space which have been measured in isotope traces, it “corrects” the MS2 spectrum annotation to the monoisotopic trace. That is why you have a high mass deviation 100 pm, but 0.0 rt tolerance. So it basically corrects the MS2 to the feature centroid that can be found/mapped by SIRIUS::preprocessing later on.

In [22]:
def preprocess(filename):
    exp = MSExperiment()
    MzMLFile().load(filename, exp)
    exp.sortSpectra(True)
    
    delta_mzs= []
    mzs = []
    rts= []
    PrecursorCorrection.correctToHighestIntensityMS1Peak(exp, 100.0, True, delta_mzs, mzs, rts)
    
    mass_traces = []
    mtd = MassTraceDetection()
    mtd_par = mtd.getDefaults()
    mtd_par.setValue("mass_error_ppm", 10.0) 
    mtd_par.setValue("noise_threshold_int", 1.0e04)
    mtd.setParameters(mtd_par)
    mtd.run(exp, mass_traces, 0)
    
    mass_traces_split = []
    mass_traces_final = []
    epd = ElutionPeakDetection()
    epd_par = epd.getDefaults()
    epd_par.setValue("width_filtering", "fixed")
    epd.setParameters(epd_par)
    epd.detectPeaks(mass_traces, mass_traces_split)
    
    if (epd.getParameters().getValue("width_filtering") == "auto"):
        epd.filterByPeakWidth(mass_traces_split, mass_traces_final)
    else:
        mass_traces_final = mass_traces_split
        
    feature_map_FFM = FeatureMap()
    feat_chrom = []
    ffm = FeatureFindingMetabo()
    ffm_par = ffm.getDefaults() 
    ffm_par.setValue("isotope_filtering_model", "none")
    ffm_par.setValue("remove_single_traces", "true")
    ffm_par.setValue("mz_scoring_by_elements", "true")
    ffm.setParameters(ffm_par)
    ffm.run(mass_traces_final, feature_map_FFM, feat_chrom)
    feature_map_FFM.setUniqueIds()
    fh = FeatureXMLFile()
    fh.store('./pyOpenMS_results/FeatureFindingMetabo.featureXML', feature_map_FFM)
    
    mfd = MetaboliteFeatureDeconvolution()
    mdf_par = mfd.getDefaults()
    mdf_par.setValue("potential_adducts",  [b"H:+:0.6",b"Na:+:0.2",b"NH4:+:0.1", b"H2O:-:0.1"])
    mdf_par.setValue("charge_min", 1, "Minimal possible charge")
    mdf_par.setValue("charge_max", 1, "Maximal possible charge")
    mdf_par.setValue("charge_span_max", 1)
    mdf_par.setValue("max_neutrals", 1)
    mfd.setParameters(mdf_par)
    
    feature_map_DEC = FeatureMap()
    cons_map0 = ConsensusMap()
    cons_map1 = ConsensusMap()
    mfd.compute(feature_map_FFM, feature_map_DEC, cons_map0, cons_map1)
    fxml = FeatureXMLFile()
    PrecursorCorrection.correctToNearestFeature(feature_map_DEC, exp, 0.0, 100.0, True, False, False, False, 3, 0)
    fxml.store("./pyOpenMS_results/deconvoluted.featureXML", feature_map_DEC)
    
    with featurexml.read("./pyOpenMS_results/deconvoluted.featureXML") as f:
        features_list = [FXML for FXML in f]
    
    df = pd.DataFrame() 

    for feat in features_list:
        idx = feat['id']
        for key in feat.keys():
            if key == 'id':
                pass
            # For col with dictionary do the following
            elif key == 'position':
                pos_list = feat['position']
                for pos in pos_list:
                    if pos['dim'] == '0':
                        df.loc[idx, 'position_0'] = pos['position']
                    elif pos['dim'] == '1':
                        df.loc[idx, 'position_1'] = pos['position']
            elif key == 'quality':
                qual_list = feat['quality']
                for qual in qual_list:
                    if qual['dim'] == '0':
                        df.loc[idx, 'quality_0'] = qual['quality']
                    elif qual['dim'] == '1':
                        df.loc[idx, 'quality_1'] = qual['quality']
            else:
                df.loc[idx, key] = feat[key]
    df_tidy = df.rename(columns = {'position_0': 'mz', 'position_1': 'RT'}, inplace = False)
    df_tidy=df_tidy.drop(columns= ["quality_0", "quality_1", "overallquality", "label", "legal_isotope_pattern"])
    df_tidy.reset_index(drop=True, inplace=True) 
    df_tidy.to_csv("./pyOpenMS_results/preprocessedDF.csv")
    return df_tidy

### SIRIUS Adapter
The SIRIUS function is optional and includes the SIRIUS Adapter Algorithm from the Boecher lab. 

This generates formula prediction from scores calculated from 1) MS2 fragmentation scores (ppm error + intensity) and 2) MS1 isotopic pattern scores.

This can help in data dereplication and analysis for direct library search of formulas. 

In [23]:
def SIRIUS(filename):    
    exp = MSExperiment()
    MzMLFile().load(filename, exp)
    exp.sortSpectra(True)
    
    delta_mzs= []
    mzs = []
    rts= []
    PrecursorCorrection.correctToHighestIntensityMS1Peak(exp, 100.0, True, delta_mzs, mzs, rts)
    sirius_algo = SiriusAdapterAlgorithm()
    sirius_algo_par = sirius_algo.getDefaults()
    sirius_algo_par.setValue("preprocessing:filter_by_num_masstraces", 2) 
    sirius_algo_par.setValue("preprocessing:precursor_mz_tolerance", 10.0) #default
    sirius_algo_par.setValue("preprocessing:precursor_mz_tolerance_unit", "ppm")
    sirius_algo_par.setValue("preprocessing:precursor_rt_tolerance", 5.0) #default
    sirius_algo_par.setValue("preprocessing:feature_only", "true")
    sirius_algo_par.setValue("sirius:profile", "orbitrap")
    sirius_algo_par.setValue("sirius:db", "none")
    sirius_algo_par.setValue("sirius:ions_considered", "[M+H]+, [M-H2O+H]+, [M+Na]+, [M+NH4]+")
    sirius_algo_par.setValue("sirius:candidates", 10)
    sirius_algo_par.setValue("sirius:elements_enforced", "CHNOS") 
    sirius_algo_par.setValue("project:processors", 2)
    sirius_algo_par.setValue("fingerid:db", "BIO")
    sirius_algo.setParameters(sirius_algo_par)
    
    featureinfo = "./pyOpenMS_results/deconvoluted.featureXML"
    fm_info = FeatureMapping_FeatureMappingInfo()
    feature_mapping = FeatureMapping_FeatureToMs2Indices() 
    sirius_algo.preprocessingSirius(featureinfo,
                                    exp,
                                    fm_info,
                                    feature_mapping)
    sirius_algo.logFeatureSpectraNumber(featureinfo, 
                                    feature_mapping,
                                    exp)
    msfile = SiriusMSFile()
    debug_level = 3
    sirius_tmp = SiriusTemporaryFileSystemObjects(debug_level)
    siriusstring= String(sirius_tmp.getTmpMsFile())
    feature_only = sirius_algo.isFeatureOnly()
    isotope_pattern_iterations = sirius_algo.getIsotopePatternIterations()
    no_mt_info = sirius_algo.isNoMasstraceInfoIsotopePattern()
    compound_info = []
    msfile.store(exp,
                 String(sirius_tmp.getTmpMsFile()),
                 feature_mapping, 
                 feature_only,
                 isotope_pattern_iterations, 
                 no_mt_info, 
                 compound_info)
    out_csifingerid = "./pyOpenMS_results/csifingerID.mzTab" 
    executable= "/Users/eeko/Desktop/software/Contents/MacOS/sirius"
    subdirs = sirius_algo.callSiriusQProcess(String(sirius_tmp.getTmpMsFile()),
                                             String(sirius_tmp.getTmpOutDir()),
                                             String(executable),
                                             String(out_csifingerid),
                                             False)
    candidates = sirius_algo.getNumberOfSiriusCandidates()
    sirius_result = MzTab()
    siriusfile = MzTabFile()
    SiriusMzTabWriter.read(subdirs,
                            filename,
                            candidates,
                            sirius_result)
    siriusfile.store("./pyOpenMS_results/out_sirius_test.mzTab", sirius_result)
    
    siriusfile= "./pyOpenMS_results/out_sirius_test.mzTab"
    sirius=  pyteomics.mztab.MzTab(siriusfile, encoding='UTF8', table_format='df')
    sirius.metadata
    df= sirius.small_molecule_table
    SIRIUS_DF= df.drop(columns= ["identifier", "smiles", "inchi_key", "description", "calc_mass_to_charge", "charge", "taxid", "species","database", "database_version", "spectra_ref", "search_engine", "modifications"])
    SIRIUS_DF.to_csv("./pyOpenMS_results/SIRIUS_DF.csv")
    return SIRIUS_DF

### CSIFinger:ID

The CSI_fingerID function is another algorithm from the Boecher lab, just like SIRIUS adapter and is using the formula predictions from SIRIUS, to search in structural libraries and predict the structure of each formula

### I need to somehow introduce the subdirs to CSI finger ID without having to rerun SIRIUS - save it somehow

In [24]:
def CSI_fingerID(filename):
    top_hits= 5
    csi_result=MzTab()
    csi_file=MzTabFile()
    CsiFingerIdMzTabWriter.read(subdirs,
                        filename,
                        top_hits,
                        csi_result)
    csi_file.store("./pyOpenMS_results/csifingerID.mzTab", csi_result)
    csi_file= "./pyOpenMS_results/csifingerID.mzTab"
    CSI=  pyteomics.mztab.MzTab(csi_file, encoding='UTF8', table_format='df')
    CSI.metadata
    df= CSI.small_molecule_table
    csifingerID= df.drop(columns= ["calc_mass_to_charge", "charge", "taxid", "species","database", "database_version", "spectra_ref", "search_engine", "modifications"])
    return csifingerID

In [16]:
filename= "./rawdata/GNPS/CollinusKirromycin.mzML"
preprocess(filename)

,mz,RT,intensity,charge,FWHM,max_height,num_of_masstraces,masstrace_intensity,masstrace_centroid_rt,masstrace_centroid_mz,isotope_distances,dc_charge_adducts,dc_charge_adduct_mass,Group,is_ungrouped_with_charge,map_idx,adducts,is_backbone
0,45.410983295999998,236.079528642847805,1.431556e07,1,1.266003,1.004315e+07,3.0,"[1.431556492448344e07, 1.385061452479889e06, 1...","[45.410983295999998, 45.410983295999998, 45.41...","[236.079528642847805, 237.083284160642108, 238...","[1.003755517794303, 0.994543795267532]",H1,1.007276,262528290233691600,1.0,NaN,NaN,NaN
1,46.649324512020002,134.05992622571415,4.488444e05,1,4.124353,3.439702e+05,2.0,"[4.488443657072215e05, 7.829680156584026e05]","[46.649324512020002, 49.121904960000002]","[134.05992622571415, 135.065193546254676]",[1.005267320540526],H1,1.007276,14532137285781141792,1.0,NaN,NaN,NaN
2,47.904151456019996,284.122961811640664,7.064417e05,1,10.355210,1.597123e+05,3.0,"[7.064417026921761e05, 8.11124627262207e05, 3....","[47.904151456019996, 54.057389008019996, 55.29...","[284.122961811640664, 285.11937656663207, 286....","[0.996414754991406, 1.019911470126317]",H1,1.007276,12666355572927274030,1.0,NaN,NaN,NaN
3,49.121904960000002,86.059963701320299,1.924124e07,1,26.451746,9.647002e+05,2.0,"[1.924124403346687e07, 7.049339221565891e05]","[49.121904960000002, 51.543910272000005]","[86.059963701320299, 87.063310928280487]",[1.003347226960187],H1,1.007276,13256968797153918491,1.0,NaN,NaN,NaN
4,49.121904960000002,102.054873314725583,1.028305e07,1,5.877229,1.949633e+06,2.0,"[1.028304551312552e07, 3.686239868791638e05]","[49.121904960000002, 49.121904960000002]","[102.054873314725583, 103.058217083983976]",[1.003343769258393],H1,1.007276,18103419384676373460,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,521.231506127999978,330.162517975732385,5.112276e05,2,4.304959,9.554357e+04,2.0,"[5.11227633050047e05, 1.138385197839318e05]","[521.231506127999978, 522.429165311999896]","[330.162517975732385, 330.664165301443234]",[0.501647325710849],H2,2.014553,8266093595435513976,1.0,NaN,NaN,NaN
885,532.090882336019945,89.008212658703414,1.543639e06,1,52.431175,5.634687e+04,2.0,"[1.543639198908519e06, 1.684513544651125e06]","[532.090882336019945, 532.090882336019945]","[89.008212658703414, 90.008334182545184]",[1.00012152384177],H1,1.007276,8775301163878956614,1.0,NaN,NaN,NaN
886,536.86878011202009,415.211431325486274,7.255946e06,1,3.394750,2.363118e+06,2.0,"[7.255945454530763e06, 1.875829088804614e06]","[536.86878011202009, 536.86878011202009]","[415.211431325486274, 416.214846619675257]",[1.003415294188983],H1,1.007276,5814284143789859866,1.0,NaN,NaN,NaN
887,602.603760559979946,358.368058191902549,9.572683e05,1,2.410447,3.431995e+05,2.0,"[9.572683956746767e05, 2.338587661306462e05]","[602.603760559979946, 602.603760559979946]","[358.368058191902549, 359.371423525032071]",[1.003365333129523],H1,1.007276,9995242384684269195,1.0,NaN,NaN,NaN


In [19]:
SIRIUS(filename)

KeyboardInterrupt: 

In [14]:
from pandas import DataFrame
import pandas as pd

import pyteomics
from pyteomics.openms import featurexml
with featurexml.read("./mzML_files/wf_testing/deconvolutedEpemicins.featureXML") as f:
    features_list = [FXML for FXML in f]
    
df = pd.DataFrame() 

for feat in features_list:
    idx = feat['id']
    for key in feat.keys():
        if key == 'id':
            pass
        # For col with dictionary do the following
        elif key == 'position':
            pos_list = feat['position']
            for pos in pos_list:
                if pos['dim'] == '0':
                    df.loc[idx, 'position_0'] = pos['position']
                elif pos['dim'] == '1':
                    df.loc[idx, 'position_1'] = pos['position']
        elif key == 'quality':
            qual_list = feat['quality']
            for qual in qual_list:
                if qual['dim'] == '0':
                    df.loc[idx, 'quality_0'] = qual['quality']
                elif qual['dim'] == '1':
                    df.loc[idx, 'quality_1'] = qual['quality']
        else:
            df.loc[idx, key] = feat[key]
df_tidy = df.rename(columns = {'position_0': 'mz', 'position_1': 'RT'}, inplace = False)
df_tidy=df_tidy.drop(columns= ["quality_0", "quality_1", "overallquality", "label", "legal_isotope_pattern"])
df_tidy.reset_index(drop=True, inplace=True) 
df_tidy


df_tidy.loc[df_tidy['mz'] == "658"]
df_tidy

,mz,RT,intensity,charge,FWHM,max_height,num_of_masstraces,masstrace_intensity,masstrace_centroid_rt,masstrace_centroid_mz,isotope_distances,Group,is_ungrouped_monoisotopic,dc_charge_adducts,dc_charge_adduct_mass,is_ungrouped_with_charge,map_idx,adducts,is_backbone,old_charge
0,39.529802865000001,349.172005427819727,6.483638e05,0,8.939569,75668.257812,1.0,[6.48363806042836e05],[39.529802865000001],[349.172005427819727],[],8497532420404919093,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,42.379768303980001,391.182592020985794,1.612096e06,0,10.832493,183479.296875,1.0,[1.612096255726351e06],[42.379768303980001],[391.182592020985794],[],1021302065514414871,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,42.379768303980001,404.190102738177075,3.534303e05,1,6.644110,129759.664062,2.0,"[3.534302611248636e05, 8.032523655859887e05]","[42.379768303980001, 42.379768303980001]","[404.190102738177075, 405.198191650642855]",[1.00808891246578],1531188304737912594,NaN,H1,1.007276,1.0,NaN,NaN,NaN,NaN
3,42.379768303980001,461.235803441382416,3.050932e05,0,4.296739,88459.125000,1.0,[3.050931746679126e05],[42.379768303980001],[461.235803441382416],[],14370752362800809058,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,43.264481632020001,361.208104366155453,2.066669e06,0,5.489381,487036.281250,1.0,[2.066669269290893e06],[43.264481632020001],[361.208104366155453],[],17717354507030679941,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14064,917.212863695999999,1321.983679608177454,2.423242e05,0,11.887800,29230.892578,1.0,[2.423241585391494e05],[917.212863695999999],[1321.983679608177454],[],16230232327793072483,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14065,918.210342559979949,420.319015627257613,2.520901e05,0,6.803956,42458.042969,1.0,[2.520900600146166e05],[918.210342559979949],[420.319015627257613],[],9712883258301259330,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14066,918.210342559979949,1421.977424770406287,1.326787e05,0,4.103276,38798.972656,1.0,[1.3267872324558e05],[918.210342559979949],[1421.977424770406287],[],3859115546576027604,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14067,918.210342559979949,1521.970953123048957,2.119047e05,0,6.773869,37788.941406,1.0,[2.119047253454692e05],[918.210342559979949],[1521.970953123048957],[],10096857489543211695,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Explanation of columns
#### mz= mass-to-charge ratio (m/z)
#### RT= retention time (min)
#### intensity = intensity of the feature (AU-arbitrary units)
#### FWHM= Full Width of the peak at Half its Maximum height
#### num_of_masstraces	= number of mass traces detected (single mass traces are excluded). This is relevant to the isotopic pattern
#### isotope_distances = distance in mz between the isotopes (jumps of app. 1 is important to confirm that this is a real feature)
#### 

In [2]:
import pandas as pd
import numpy as np
import sys
import pyteomics
from pyteomics import mztab
filename= "./mzML_files/wf_testing/out_sirius_testGermB.mzTab"
sirius=  pyteomics.mztab.MzTab(filename, encoding='UTF8', table_format='df')
sirius.metadata
df= sirius.small_molecule_table
data= df.drop(columns= ["identifier", "smiles", "inchi_key", "description", "calc_mass_to_charge", "charge", "taxid", "species","database", "database_version", "spectra_ref", "search_engine", "modifications"])
data

,chemical_formula,exp_mass_to_charge,retention_time,best_search_engine_score[1],best_search_engine_score[2],best_search_engine_score[3],opt_global_adduct,opt_gobal_precursorFormula,opt_global_rank,opt_global_explainedPeaks,opt_global_explainedIntensity,opt_global_median_mass_error_fragment_peaks_ppm,opt_global_median_absolute_mass_error_fragment_peaks_ppm,opt_global_mass_error_precursor_ppm,opt_global_compoundId,opt_global_compoundScanNumber,opt_global_featureId,opt_global_native_id
0,CH4O4,81.017307,262.347736,10.294010,8.832051,1.461959,[M + H]+,CH4O4,1,2,0.008202,-13.847945,13.847945,-11.453751,1286,1287,id_15401598686271704206,controllerType=0 controllerNumber=1 scan=1287|...
1,C2H5FS,81.017307,262.347736,1.771033,1.771033,0.000000,[M + H]+,C2H5FS,2,2,0.008202,4.838726,4.838726,5.323257,1286,1287,id_15401598686271704206,controllerType=0 controllerNumber=1 scan=1287|...
2,CB2O3,81.017307,262.347736,-1.872022,-1.872022,0.000000,[M + H]+,CB2O3,3,1,0.000000,-7.247884,7.247884,-7.247884,1286,1287,id_15401598686271704206,controllerType=0 controllerNumber=1 scan=1287|...
3,C5H12OS,121.068635,300.520158,14.094925,11.656118,2.438807,[M + H]+,C5H12OS,1,4,0.092262,3.687210,4.694264,3.906110,1539,1540,id_15868649392726213180,controllerType=0 controllerNumber=1 scan=1540|...
4,C5H8B2S,121.068635,300.520158,8.513833,8.513833,0.000000,[M + H]+,C5H8B2S,2,5,0.110982,6.720613,9.794502,6.720613,1539,1540,id_15868649392726213180,controllerType=0 controllerNumber=1 scan=1540|...
5,C5H8F2N,121.068635,300.520158,1.258188,1.258188,0.000000,[M + H]+,C5H8F2N,3,3,0.074048,-14.698028,14.698028,-9.265691,1539,1540,id_15868649392726213180,controllerType=0 controllerNumber=1 scan=1540|...
6,C5H6BN2O,121.068635,300.520158,-1.303280,-1.303280,0.000000,[M + H]+,C5H6BN2O,4,2,0.052254,4.027006,4.027006,3.379071,1539,1540,id_15868649392726213180,controllerType=0 controllerNumber=1 scan=1540|...
7,CH8B2O5,121.068635,300.520158,-2.279097,-2.279097,0.000000,[M + H]+,CH8B2O5,5,1,0.000000,-13.946176,13.946176,-13.946176,1539,1540,id_15868649392726213180,controllerType=0 controllerNumber=1 scan=1540|...
8,C6H13FN4,183.102896,418.423143,53.600507,53.600507,0.000000,[M + Na]+,C6H13FN4,1,20,0.340086,-4.561994,7.225714,6.830271,2228,2229,id_14840376333216993323,controllerType=0 controllerNumber=1 scan=2229|...
9,C11H14N,183.102896,418.423143,23.223615,20.174189,3.049426,[M + Na]+,C11H14N,2,11,0.201965,-7.280660,7.621158,5.739016,2228,2229,id_14840376333216993323,controllerType=0 controllerNumber=1 scan=2229|...


In [4]:
filename= "./mzML_files/wf_testing/csifingerID.mzTab"
CSI=  pyteomics.mztab.MzTab(filename, encoding='UTF8', table_format='df')
CSI.metadata
df= CSI.small_molecule_table
csifingerID= df.drop(columns= ["calc_mass_to_charge", "charge", "taxid", "species","database", "database_version", "spectra_ref", "search_engine", "modifications"])
csifingerID

,identifier,chemical_formula,smiles,inchi_key,description,exp_mass_to_charge,retention_time,best_search_engine_score[1],opt_global_rank,opt_global_compoundId,opt_global_compoundScanNumber,opt_global_featureId,opt_global_native_id,opt_global_adduct,opt_global_dblinks,opt_global_dbflags
0,123571071,C17H25BN2O2S,B1(OC(C(O1)(C)C)(C)C)C2=CC3=C(C=C2)N(SN3C)CC4CC4,QCQPLSTXPJUAAM,None,349.209026,123.805954,-260.201500,1,745,746,id_6128946280250909851,controllerType=0 controllerNumber=1 scan=746,[M + H3N + H]+,PubChem:(123571071),2
1,131432082,C17H28BN3O2S,B1(OC(C(O1)(C)C)(C)C)C2=CN=C(S2)N3CCN4CCCCC4C3,SECXVOZRGXMKEM,None,349.209026,123.805954,-299.350229,1,745,746,id_6128946280250909851,controllerType=0 controllerNumber=1 scan=746,[M + H]+,PubChem:(131432082),2
2,109567509,C15H31FN4O3S,CCNC(=NCCCF)N1CCN(CC1)S(=O)(=O)CCOC(C)C,BRHUEMIRBTXUSW,None,349.209026,123.805954,-267.945346,1,745,746,id_6128946280250909851,controllerType=0 controllerNumber=1 scan=746,[M - H2O + H]+,PubChem:(109567509),2
3,70910984,C15H26FN3O2S,CC(C)(C)S(=O)(=O)NCCCCCNC1=NC=C(C=C1)CF,PEYBFSDPVVZPQL,None,349.209026,123.805954,-244.285143,1,745,746,id_6128946280250909851,controllerType=0 controllerNumber=1 scan=746,[M + H3N + H]+,PubChem:(70910984),2
4,128994375,C15H26FN3O2S,CC1=NC(=CS1)CN2CC(CC2CN(C)CC(COC)O)F,CQZCDUGNYLCVRO,None,349.209026,123.805954,-279.329205,2,745,746,id_6128946280250909851,controllerType=0 controllerNumber=1 scan=746,[M + H3N + H]+,PubChem:(128994375),2
5,138722980,C11H17FN6O3,CC(C(CO)OC(CF)N1C=NC2=C(N=C(N=C21)N)N)O,JJXXHWZKDWLJHM,None,323.122389,356.847436,-270.225119,1,2846,2847,id_11377481446250170993,controllerType=0 controllerNumber=1 scan=2847,[M + Na]+,PubChem:(138722980),2
6,135244532,C11H17FN6O3,CC1C(N2C(=N1)C(=NC(=N2)N)N)C3C(C(C(O3)CO)O)F,VNYAFPCEJVZLAO,None,323.122389,356.847436,-299.163846,2,2846,2847,id_11377481446250170993,controllerType=0 controllerNumber=1 scan=2847,[M + Na]+,PubChem:(135244532),2
7,137397861|137397935,C11H17FN6O3,COC(CN)(CO)C(C(N1C=NC2=C(N=CN=C21)N)F)O,RQTBEDYRRNIRGZ,None,323.122389,356.847436,-314.805122,3,2846,2847,id_11377481446250170993,controllerType=0 controllerNumber=1 scan=2847,[M + Na]+,PubChem:(137397861 137397935),2
8,118595172,C11H17FN6O3,C=C1NC(C=CN1C2C(C(C(O2)(CN=[N+]=[N-])CO)O)F)N,MYSZRXFXRIVGPS,None,323.122389,356.847436,-438.401003,4,2846,2847,id_11377481446250170993,controllerType=0 controllerNumber=1 scan=2847,[M + Na]+,PubChem:(118595172),2
9,25000395,C11H20BN3O4S,B1(OC(C(O1)(C)C)(C)C)C2=CN(C=N2)S(=O)(=O)N(C)C,ZWQXNFBXPCSFAO,"1-(N,N-Dimethylsulfamoyl)imidazole-4-boronic a...",323.122389,356.847436,-259.475588,1,2846,2847,id_11377481446250170993,controllerType=0 controllerNumber=1 scan=2847,[M + Na]+,PubChem:(25000395),2
